Axioms for the theory of <a class="ProveItLink" href="theory.ipynb">proveit.physics.quantum.QPE</a>
========

In [ ]:
import proveit
# Prepare this notebook for defining the axioms of a theory:
%axioms_notebook # Keep this at the top following 'import proveit'.
from proveit import defaults
defaults.automation = True # Hack for ExprRange simplification to go through
from proveit import Literal, ExprArray, ExprRange, ExprTuple
from proveit import a, b, c, d, e, eps, k, l, m, n, r, t, U
# from proveit.linalg import MatrixProd, ScalarProd, TensorExp, TensorProd
from proveit.linear_algebra import MatrixMult, ScalarMult, TensorExp, TensorProd
from proveit.logic import Equals, Forall, NotEquals, InSet, Set
from proveit.numbers import (zero, one, two, three, i, pi, Div, Exp,
                             Integer, Interval, IntervalCO, Natural, NaturalPos, Neg)
from proveit.numbers import Add, exp, Floor, frac, LessEq, Mod, ModAbs, Mult, Prod, subtract, Sum
from proveit.physics.quantum import (
    Ket, Meas, NumBra, NumKet, RegisterSU, Qmult,
    QubitRegisterSpace, ket0, ket1)
#from proveit.physics.quantum.circuit import (Circuit, Input, Output, Gate, MultiQubitGate, 
#                                             IdentityOp, MultiWire, CircuitEquiv)
from proveit.physics.quantum.circuits import QcircuitEquiv
from proveit.physics.quantum.QPE import (
    _alpha_l, _b, _delta, _m, _n, _t, _phase, _phase_est, _phase_m, _psi_t, _Psi,
    _two_pow_t, two_pow_t, _u, _u_tilde, _U, _U_pow_two_pow_k,
    #p_0, p_k, p_r, p_prime, p_prime_0, p_prime_k, p_prime_r, psi_prime, psi_prime_1, psi_prime_t,
    _QPE_U_t, _QPE1_U_t, _QPE_U_t_circuit, QPE1_U_t_circuit, 
    _psi_t_circuit, _Psi_circuit, _phase_est_circuit, _success_prob_e)
from proveit.physics.quantum.QPE.phase_est_ops import ModAdd, Pfail, Psuccess, SubIndexed
from proveit.physics.quantum.QFT import InverseFourierTransform
from proveit.statistics import Prob

In [ ]:
defaults.automation = False # Hack for ExprRange simplification to go through
%begin axioms
defaults.automation = True # Hack for ExprRange simplification to go through
print("We need automation to simplify ExprRanges in Qcircuits")

## Local axioms (for convenience - used only internally)

Let $U$ be a unitary operator that acts on $m$ qubits, with $\lvert u\rangle$ as an eigenstate of $U$ with eigenvalue $e^{2 \pi i \varphi}$:

In [ ]:
_m_in_nat_pos = InSet(_m, NaturalPos)

In [ ]:
_unitary_U = InSet(_U, RegisterSU(_m))

In [ ]:
_u_ket_register = InSet(Ket(_u), QubitRegisterSpace(_m))

In [ ]:
_u_tilde_ket_register = InSet(Ket(_u_tilde), QubitRegisterSpace(_m))

In [ ]:
_phase_in_interval = InSet(_phase, IntervalCO(zero, one))

In [ ]:
_eigen_uu = Equals(MatrixMult(_U, Ket(_u)), 
                   ScalarMult(exp(Mult(two, pi, i, _phase)), Ket(_u)))

In [ ]:
# t (represented by the Literal t_)
# denotes the number of Qbits in the input register
_t_in_natural_pos = InSet(_t, NaturalPos)

In [ ]:
_psi_t_kets_in_qreg_space = Forall(t, InSet(Ket(_psi_t), 
                                         QubitRegisterSpace(t)), 
                                domain=NaturalPos)

In [ ]:
_Psi_ket_in_qreg_space = InSet(Ket(_Psi), QubitRegisterSpace(_t))

In [ ]:
_psi_t_def = Forall(t, _psi_t_circuit.literals_as_variables(_t), 
                    domain=NaturalPos)

In [ ]:
_Psi_def = _Psi_circuit

In [ ]:
_phase_est_def = _phase_est_circuit

Let $\lvert \Psi \rangle$ be the outcome of the $t$-qubit register of ${\rm QPE}(U, t)$ acting on $\lvert u \rangle$, $m$ be a random variable representing the measurement of Psi with the register interpreted as an integer (via binary representation), and $\varphi_m = 2 \pi m/2^t$ be the random variable phase outcome of ${\rm QPE}(U, t)$:

In [ ]:
# _m_def = Equals(_m, Meas(Ket(_Psi)))

In [ ]:
_m_def = Equals(_m, Mult(_two_pow_t, _phase_est))

In [ ]:
# _phase_m_def = Equals(_phase_m, frac(_m, _two_pow_t))

Let $b$ be the value for $m$ that gives the closest $\varphi_m$ to $\varphi$ without exceeding it, and let $\delta$ be this difference:

In [ ]:
_best_def = Equals(_b, Floor(Mult(_phase, _two_pow_t)))

In [ ]:
_delta_def = Equals(_delta, subtract(_phase, frac(_b, _two_pow_t)))

The probability of success is defined as $\theta_m$ being within some epsilon of $\theta$: 

In [ ]:
_success_def = Forall(
        e,
        Equals(Psuccess(e),
               _success_prob_e),
        domain=NaturalPos)

In [ ]:
_fail_def = Forall(
    e,
    Equals(Pfail(e), subtract(one, Psuccess(e))),
    domain=NaturalPos)

Let $\alpha_l$ be the amplitude of $\lvert \Psi \rangle$ for the $b \oplus l$ state, where $\oplus$ is defined as addition modulo $2^t$:

In [ ]:
_mod_add_def = Forall(
    (a, b),
    Equals(ModAdd(a, b), Mod(Add(a, b), _two_pow_t)),
    domain=Integer)

In [ ]:
_alpha_def = Forall(
    l,
    Equals(_alpha_l,
           Qmult(NumBra(ModAdd(_b, l), _t),
                 Ket(_Psi))),
    domain=Integer)

### The externally usable axioms (as opposed to local axioms)

These assumptions are needed for proper formatting of `QPE1_U_t_circuit`.

In [ ]:
from proveit.numbers import greater
defaults.assumptions = [greater(t, two), greater(subtract(t, one), zero)]

QPE1_def = Forall(
    (m, t), Forall(
        U, QcircuitEquiv(_QPE1_U_t.literals_as_variables(_U, _t, _m), 
                         QPE1_U_t_circuit),
        domain=RegisterSU(m)),
    domain=NaturalPos)

In [ ]:
QPE_def = Forall(
    (m, t), Forall(
        U, QcircuitEquiv(_QPE_U_t.literals_as_variables(_U, _t, _m), 
                         _QPE_U_t_circuit.literals_as_variables(_U, _t, _m)),
        domain=RegisterSU(m)),
    domain=NaturalPos)

In [ ]:
%end axioms